In [1]:
import os
import torch
from torch import nn
import dltools

In [4]:
class TokenEmbedding:
    def __init__(self, file_path):
        self.idx_to_token,self.idx_to_vec = self._loda_embedding(file_path)
        self.unknown_idx = 0
        self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_token)}
    def _loda_embedding(self,file_path):
        idx_to_token,idx_to_vec = ['<unk>'],[]
        with open(file_path,'r',encoding='utf-8') as f:
            for line in f:
                elems = line.rstrip().split(' ')
                token,elems = elems[0] ,[float(elem) for elem in elems[1:]]
                if len(elems) > 1:
                    idx_to_token.append(token)
                    idx_to_vec.append(elems)
        idx_to_vec=[[0] * len(idx_to_vec[0])] + idx_to_vec
        return idx_to_token, torch.tensor(idx_to_vec) 
    def __getitem__(self,tokens):
        indices = [self.token_to_idx.get(token,self.unknown_idx) for token in tokens]
        vecs  = self.idx_to_vec[torch.tensor(indices)]
        return vecs
    
    def __len__(self):
        return len(self.idx_to_token)

In [5]:
glove_6b50d = TokenEmbedding(r'D:/ai/hello_nlp\data/glove.6B/glove.6B.50d.txt')

In [25]:
len(glove_6b50d)

999995

In [26]:
glove_6b50d.token_to_idx['bear']

3326

In [27]:
glove_6b50d.idx_to_token[2932]

'correctly'

In [28]:
def knn(W,x,k):
    cos = torch.mv(W,x.reshape(-1,)) / (torch.sqrt((torch.sum(W*W,axis=1)) + 1e-9) * torch.sqrt((x * x).sum()))
    _, topk = torch.topk(cos,k=k)
    return topk,[cos[int(i)] for i in topk]


In [13]:
def get_similar_tokens(query_token,k,embed):
    topk,cos = knn(embed.idx_to_vec,embed[[query_token]],k+1)
    for  i,c in zip(topk[1:],cos[1:]):
        print(f'{embed.idx_to_token[int(i)]}:cos相似度={float(c):.3f}')

In [29]:
get_similar_tokens('chip',8,glove_6b50d)

chips:cos相似度=0.829
microchip:cos相似度=0.649
silicon:cos相似度=0.643
microprocessor:cos相似度=0.619
semiconductor:cos相似度=0.590
barganing:cos相似度=0.588
shoulder:cos相似度=0.581
circuitry:cos相似度=0.581


In [17]:
def get_analogy(token_a,token_b,token_c,embed):
    vecs = embed[[token_a,token_b,token_c]]
    x = vecs[1] - vecs[0] +vecs[2]
    topk,cos = knn(embed.idx_to_vec,x,1)
    return embed.idx_to_token[int(topk[0])]

In [34]:
get_analogy('man','woman','boy',glove_6b50d)

'girl'

In [23]:
glove_6b50d = TokenEmbedding(r'D:/ai/hello_nlp/data/wiki-news-300d-1M.vec/wiki-news-300d-1M.vec')

In [24]:
len(glove_6b50d)

999995